# JS: Core

> Master composer for card stack JavaScript. Combines viewport height,
> scroll navigation, touch navigation, page navigation,
> width/scale/count management, and the master HTMX coordinator into a
> single namespaced IIFE.

In [ ]:
#| default_exp js.core

In [ ]:
#| export
from typing import Any, Optional, Tuple

from fasthtml.common import Script

from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds
from cjm_fasthtml_card_stack.core.models import CardStackUrls, CardStackState
from cjm_fasthtml_card_stack.core.constants import (
    width_storage_key, scale_storage_key, card_count_storage_key,
    auto_count_storage_key,
    DEFAULT_CARD_WIDTH, DEFAULT_CARD_SCALE, DEFAULT_VISIBLE_COUNT,
)
from cjm_fasthtml_card_stack.js.viewport import generate_viewport_height_js
from cjm_fasthtml_card_stack.js.scroll import generate_scroll_nav_js
from cjm_fasthtml_card_stack.js.touch import generate_touch_nav_js
from cjm_fasthtml_card_stack.js.navigation import generate_page_nav_js
from cjm_fasthtml_card_stack.js.controls import (
    _generate_width_mgmt_js, _generate_scale_mgmt_js, _generate_card_count_mgmt_js,
)
from cjm_fasthtml_card_stack.js.auto_adjust import _generate_auto_adjust_js

## Master Coordinator

Applies all viewport settings (width, height, scroll, touch) then reveals
the viewport using a double-RAF pattern. Also handles HTMX afterSettle
events.

In [ ]:
#| export
def _generate_coordinator_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config for prefix-unique listener guards
    focus_position: Optional[int] = None,  # Focus slot offset (None=center, -1=bottom, 0=top)
) -> str:  # JS code fragment for master coordinator
    """Generate JS for the master coordinator and HTMX listener."""
    handler_key = f"_csHandlers_{config.prefix.replace('-', '_')}"
    js_focus_pos = "null" if focus_position is None else str(focus_position)
    return f"""
        // === Grid Template Management ===
        ns.applyGridTemplate = function() {{
            const inner = document.getElementById('{ids.card_stack_inner}');
            if (!inner) return;
            const focusPosRaw = {js_focus_pos};
            let tmpl;
            if (focusPosRaw === null) {{
                tmpl = '1fr auto 1fr';
            }} else if (focusPosRaw === 0) {{
                tmpl = 'auto 1fr';
            }} else if (focusPosRaw < 0) {{
                tmpl = '1fr auto';
            }} else {{
                tmpl = '1fr auto 1fr';
            }}
            inner.style.gridTemplateRows = tmpl;
        }};

        // === Master Coordinator ===
        ns.applyAllViewportSettings = function() {{
            requestAnimationFrame(function() {{
                if (ns.applyWidth) ns.applyWidth();
                if (ns.applyScale) ns.applyScale();
                if (ns.applyGridTemplate) ns.applyGridTemplate();
                if (ns.recalculateHeight) ns.recalculateHeight();

                if (ns._setupScrollNav) ns._setupScrollNav();
                if (ns._setupTouchNav) ns._setupTouchNav();

                requestAnimationFrame(function() {{
                    const cs2 = document.getElementById('{ids.card_stack}');
                    if (cs2) cs2.style.opacity = '1';

                    // Continue auto-adjust loop if an adjustment is in flight
                    if (typeof _autoAdjusting !== 'undefined' && _autoAdjusting) {{
                        _autoAdjusting = false;
                        requestAnimationFrame(function() {{
                            if (ns._runAutoAdjust) ns._runAutoAdjust();
                        }});
                    }}
                }});
            }});
        }};

        // === HTMX Event Listeners ===
        // Remove old listeners from previous IIFE (handles HTMX page navigation
        // that re-executes this script without a full page reload).
        if (window.{handler_key}) {{
            document.body.removeEventListener('htmx:afterSwap', window.{handler_key}.swap);
            document.body.removeEventListener('htmx:afterSettle', window.{handler_key}.settle);
        }}

        function _afterSwapHandler(evt) {{
            const target = evt.detail.target;
            if (!target) return;
            const cs = document.getElementById('{ids.card_stack}');
            const isCSSwap = (
                target.id === '{ids.card_stack}' ||
                target.id === '{ids.card_stack_inner}' ||
                (cs && cs.contains(target))
            );
            if (isCSSwap && typeof _autoGrowing !== 'undefined' && _autoGrowing) {{
                _hideNewItems();
            }}
        }}

        function _afterSettleHandler(evt) {{
            const target = evt.detail.target;
            if (!target) return;
            const cs = document.getElementById('{ids.card_stack}');
            const isCSSwap = (
                target.id === '{ids.card_stack}' ||
                target.id === '{ids.card_stack_inner}' ||
                (cs && cs.contains(target))
            );
            if (isCSSwap) {{
                _syncCountDropdown();
                ns.applyAllViewportSettings();
            }}
        }}

        window.{handler_key} = {{ swap: _afterSwapHandler, settle: _afterSettleHandler }};
        document.body.addEventListener('htmx:afterSwap', _afterSwapHandler);
        document.body.addEventListener('htmx:afterSettle', _afterSettleHandler);

        // === Initialize ===
        requestAnimationFrame(function() {{
            _syncCountDropdown();
            setTimeout(function() {{
                // Scroll to top before height calculation to ensure consistent
                // viewport-relative measurements. HTMX navigation may preserve
                // scroll position from the previous page, causing incorrect height.
                window.scrollTo(0, 0);
                ns.applyAllViewportSettings();
                // Trigger auto-adjust after initial layout settles
                if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
            }}, 50);
        }});
    """

## Global Callback Registration

The `cjm-fasthtml-keyboard-navigation` library looks up JS callbacks via
`window[action.jsCallback]()`. Since our functions live on the namespaced
`ns` object, we register global wrappers that delegate to the namespace.

In [ ]:
#| export
# Callback name constants — must match what create_card_stack_nav_actions uses
_GLOBAL_CALLBACKS = (
    "jumpPageUp",
    "jumpPageDown",
    "jumpToFirstItem",
    "jumpToLastItem",
    "decreaseWidth",
    "increaseWidth",
    "decreaseScale",
    "increaseScale",
)

def global_callback_name(
    prefix: str,  # Card stack instance prefix
    callback: str,  # Base callback name (e.g., "jumpPageUp")
) -> str:  # Global function name (e.g., "cs0_jumpPageUp")
    """Generate a prefix-unique global callback name for keyboard navigation."""
    return f"{prefix}_{callback}"

def _generate_global_callbacks_js(
    config: CardStackConfig,  # Config with prefix
) -> str:  # JS code fragment registering global wrappers
    """Register global wrappers for keyboard navigation system."""
    lines = ["        // === Global Keyboard Callbacks ==="]
    for cb in _GLOBAL_CALLBACKS:
        global_name = global_callback_name(config.prefix, cb)
        lines.append(f"        window['{global_name}'] = function() {{ if (ns.{cb}) ns.{cb}(); }};")
    return "\n".join(lines)

## generate_card_stack_js

The main entry point that composes all JS fragments into a single namespaced
IIFE. The consumer calls this once in their step renderer.

In [ ]:
#| export
def generate_card_stack_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    button_ids: CardStackButtonIds,  # Button IDs for keyboard triggers
    config: CardStackConfig,  # Card stack configuration
    urls: CardStackUrls,  # URL bundle for routing
    container_id: str = "",  # Consumer's parent container ID (for height calc)
    extra_scripts: Tuple[str, ...] = (),  # Additional JS to include in the IIFE
    focus_position: Optional[int] = None,  # Focus slot offset (None=center, -1=bottom, 0=top)
) -> Any:  # Script element with all card stack JavaScript
    """Compose all card stack JS into a single namespaced IIFE."""
    prefix = config.prefix
    extra_js = "\n".join(extra_scripts)

    # Collect all fragments
    viewport_js = generate_viewport_height_js(ids, container_id)
    scroll_js = generate_scroll_nav_js(ids, button_ids, config.disable_scroll_in_modes)
    touch_js = generate_touch_nav_js(ids, button_ids, config.disable_scroll_in_modes)
    page_nav_js = generate_page_nav_js(button_ids)
    width_js = _generate_width_mgmt_js(ids, config, urls)
    scale_js = _generate_scale_mgmt_js(ids, config, urls)
    count_js = _generate_card_count_mgmt_js(ids, config, urls)
    auto_js = _generate_auto_adjust_js(ids, config, urls, focus_position) if config.auto_visible_count else ""
    global_cbs_js = _generate_global_callbacks_js(config)
    coordinator_js = _generate_coordinator_js(ids, config, focus_position)

    return Script(f"""(function() {{
        window.cardStacks = window.cardStacks || {{}};
        const ns = window.cardStacks['{prefix}'] = {{}};

        {viewport_js}
        {scroll_js}
        {touch_js}
        {page_nav_js}
        {width_js}
        {scale_js}
        {count_js}
        {auto_js}
        {global_cbs_js}
        {coordinator_js}
        {extra_js}
    }})();""")

In [ ]:
from cjm_fasthtml_card_stack.core.config import _reset_prefix_counter

In [ ]:
# Test setup: shared fixtures for composition tests
_reset_prefix_counter()
config = CardStackConfig()
ids = CardStackHtmlIds(prefix=config.prefix)
btn = CardStackButtonIds(prefix=config.prefix)
urls = CardStackUrls(
    nav_up="/cs/nav_up", nav_down="/cs/nav_down",
    nav_first="/cs/nav_first", nav_last="/cs/nav_last",
    nav_page_up="/cs/nav_page_up", nav_page_down="/cs/nav_page_down",
    nav_to_index="/cs/nav_to_index",
    update_viewport="/cs/update_viewport",
    save_width="/cs/save_width", save_scale="/cs/save_scale",
)

script = generate_card_stack_js(ids, btn, config, urls, container_id="my-app")
js_text = script.children[0] if script.children else ""

# Namespace setup
assert "window.cardStacks" in js_text
assert f"'{config.prefix}'" in js_text

# All sections present in composed output
for section in [
    "Viewport Height", "Scroll Navigation", "Touch Navigation",
    "Page Navigation", "Width Management", "Scale Management",
    "Card Count Management", "Auto Visible Count Adjustment",
    "Grid Template Management", "Global Keyboard Callbacks",
    "Master Coordinator", "HTMX Event Listeners",
]:
    assert section in js_text, f"Missing section: {section}"

# Scroll-to-top before height calculation (fixes HTMX navigation scroll position issue)
assert "window.scrollTo(0, 0)" in js_text, "Missing scroll-to-top in initialization"

print("Composition: namespace and section presence tests passed!")

Composition: namespace and section presence tests passed!


In [ ]:
# Test key namespace functions are exposed
for fn in [
    "ns.updateWidth", "ns.updateScale", "ns.updateCardCount",
    "ns.handleCountChange", "ns._autoUpdateCount", "ns._runAutoAdjust",
    "ns.triggerAutoAdjust", "ns.applyGridTemplate", "ns.jumpToFirstItem",
    "ns.jumpToLastItem", "ns.applyAllViewportSettings", "ns.recalculateHeight",
    "ns.decreaseWidth", "ns.increaseWidth", "ns.decreaseScale", "ns.increaseScale",
    "ns._setupScrollNav", "ns._setupTouchNav",
]:
    assert fn in js_text, f"Missing function: {fn}"

# Touch navigation (Pointer Events API)
assert "pointerdown" in js_text
assert "pointermove" in js_text
assert "setPointerCapture" in js_text
assert "momentumTick" in js_text
print("Composition: namespace function exposure tests passed!")

Composition: namespace function exposure tests passed!


In [ ]:
# Test auto-adjust and growth validation in composed output
assert "_autoAdjusting" in js_text
assert "_autoGrowing" in js_text
assert "_preGrowthItemIds" in js_text
assert "_snapshotItemIds" in js_text
assert "_hideNewItems" in js_text
assert "_revealNewItems" in js_text
assert "_validateGrowth" in js_text
assert "ns._cancelAutoGrowth" in js_text
assert "htmx:afterSwap" in js_text

# Card count uses swap:'none' for OOB updates
assert "swap: 'none',\n                    values: { visible_count:" in js_text
print("Composition: auto-adjust and growth validation tests passed!")

Composition: auto-adjust and growth validation tests passed!


In [ ]:
# Test global callback wrappers and HTMX listener cleanup
prefix = config.prefix
for cb in ["jumpPageUp", "jumpToFirstItem", "decreaseWidth", "increaseWidth",
           "decreaseScale", "increaseScale"]:
    assert f"window['{prefix}_{cb}']" in js_text, f"Missing global callback: {cb}"

# HTMX listeners use remove-and-replace pattern
handler_key = f"_csHandlers_{config.prefix.replace('-', '_')}"
assert f"window.{handler_key}" in js_text
assert "removeEventListener" in js_text
assert "_afterSwapHandler" in js_text
assert "_afterSettleHandler" in js_text
print("Composition: global callbacks and HTMX listener tests passed!")

Composition: global callbacks and HTMX listener tests passed!


In [ ]:
# Test focus_position parameter produces correct JS literals
_reset_prefix_counter()
_fp_config = CardStackConfig()
_fp_ids = CardStackHtmlIds(prefix=_fp_config.prefix)
_fp_btn = CardStackButtonIds(prefix=_fp_config.prefix)

# Default (None) → JS null (center focus)
_fp_script = generate_card_stack_js(_fp_ids, _fp_btn, _fp_config, urls)
_fp_js = _fp_script.children[0] if _fp_script.children else ""
assert "const focusPosRaw = null;" in _fp_js

# Bottom focus (-1)
_fp_script_bottom = generate_card_stack_js(_fp_ids, _fp_btn, _fp_config, urls, focus_position=-1)
_fp_js_bottom = _fp_script_bottom.children[0] if _fp_script_bottom.children else ""
assert "const focusPosRaw = -1;" in _fp_js_bottom

# Top focus (0)
_fp_script_top = generate_card_stack_js(_fp_ids, _fp_btn, _fp_config, urls, focus_position=0)
_fp_js_top = _fp_script_top.children[0] if _fp_script_top.children else ""
assert "const focusPosRaw = 0;" in _fp_js_top

print("Focus position parameter tests passed!")

Focus position parameter tests passed!


In [ ]:
# Test extra_scripts injection
_reset_prefix_counter()
config2 = CardStackConfig()
ids2 = CardStackHtmlIds(prefix=config2.prefix)
btn2 = CardStackButtonIds(prefix=config2.prefix)

script2 = generate_card_stack_js(
    ids2, btn2, config2, urls,
    extra_scripts=("ns.customFunction = function() { console.log('custom'); };",)
)
js2 = script2.children[0] if script2.children else ""
assert "ns.customFunction" in js2
print("Extra scripts injection test passed!")

Extra scripts injection test passed!


In [ ]:
# Test with disable_scroll_in_modes
config3 = CardStackConfig(prefix="split-test", disable_scroll_in_modes=("split",))
ids3 = CardStackHtmlIds(prefix=config3.prefix)
btn3 = CardStackButtonIds(prefix=config3.prefix)

script3 = generate_card_stack_js(ids3, btn3, config3, urls)
js3 = script3.children[0] if script3.children else ""
assert "isScrollDisabled" in js3
assert "isTouchDisabled" in js3
assert "'split'" in js3
print("Scroll/touch mode disabling in composed JS test passed!")

# Test with auto_visible_count=False excludes auto-adjust and growth validation
config4 = CardStackConfig(prefix="no-auto", auto_visible_count=False)
ids4 = CardStackHtmlIds(prefix=config4.prefix)
btn4 = CardStackButtonIds(prefix=config4.prefix)

script4 = generate_card_stack_js(ids4, btn4, config4, urls)
js4 = script4.children[0] if script4.children else ""
assert "Auto Visible Count Adjustment" not in js4
assert "_snapshotItemIds" not in js4
# assert "_hideNewItems" not in js4
assert "_revealNewItems" not in js4
# But handleCountChange and _isAutoMode are still in count management
assert "ns.handleCountChange" in js4
print("auto_visible_count=False exclusion test passed!")

Scroll/touch mode disabling in composed JS test passed!
auto_visible_count=False exclusion test passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()